In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
project_path = r"/content/drive/MyDrive/slca-main-pavan"
#project_path = r"/content/drive/MyDrive/Colab-Notebooks/slca_project/slca-main-pavan"

In [6]:
 %cd "/content/drive/MyDrive/slca-main-pavan/src"
#%cd "/content/drive/MyDrive/Colab-Notebooks/slca_project/slca-main-pavan/src"

/content/drive/MyDrive/slca-main-pavan/src


In [7]:
run_command = f"{project_path}/src/slca.py"
%run {run_command}

[[1.  0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 1.  0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 1.  0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 1.  0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 1.  0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 1.  0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 1.  0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 1. ]]


In [8]:
run_command = f"genetic.py"
# run_command = f"{project_path}/src/genetic.py"
%run {run_command}

[[1.  0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 1.  0.5 0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 1.  0.5 0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 1.  0.5 0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 1.  0.5 0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 1.  0.5 0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 1.  0.5]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 1. ]]


In [9]:
run_command = f"utils.py"
%run {run_command}

In [10]:
%ls ../config

ga_parameters.json          slca_parameters_init.json   slca_parameters_sim.json
slca_parameters_fixed.json  slca_parameters_range.json


In [11]:
#%ls /content/drive/MyDrive/Colab-Notebooks/slca_project/slca-main-pavan/data/smaps/eml-net1/childroom01_smap.jpg
%ls /content/drive/MyDrive/slca-main-pavan/data/smaps/eml-net1/childroom01_smap.png

/content/drive/MyDrive/slca-main-pavan/data/smaps/eml-net1/childroom01_smap.png


In [12]:
%rm -rf /content/output/*

# Genetic

In [13]:
from google.colab import output
import numpy as np
import scipy.stats as stats
import scipy.signal as signal
from scipy.stats import anderson
from scipy.ndimage import gaussian_filter
import multiprocessing as mp
import json
import logging
import os
from slca import *
from utils import *

from matplotlib.image import imread
import matplotlib.pyplot as plt
import seaborn as sns

class GA():


    def __init__(self, lca_model, n_units, time, params_diap, human_rt_path, human_coords_path, fixed,stim_path,params_range,part,pop_size=1,gen_size=3):
        self.params_diap = params_diap
        self.time = time
        self.n_units = n_units

        self.dt_t = 0.1
        self.threshold = 5.0

        self.n_desc = int(gen_size / 4)

        self.human_rt = self.load_human_data(human_rt_path)
        self.human_coords = self.get_human_gaze(human_coords_path)
        self.stim = self.load_stim(stim_path)
        self.fixed = fixed
        self.params_range = params_range
        self.unfixed_inds = tuple(set(range(len(self.params_range))) - set(fixed.keys()))
        self.part = part
        self.lca_model = lca_model
        self.output = output
        self.gen_size = gen_size




    def load_human_data(self, data_path):
        with open(data_path, 'r') as f:
            data = json.load(f)
        return data

    def get_human_gaze(self, human_coords_path):
        human_coords = self.load_human_data(human_coords_path)
        data = {}
        for stim in human_coords:
            data[stim] = {}

            for participant in human_coords[stim]:
                gazepoints = human_coords[stim][participant]
                if len(gazepoints) > 0:
                    lens = self.human_rt[stim][participant]
                    gaze = np.zeros((68, 120))
                    for i in range(len(gazepoints)):
                        y, x = gazepoints[i]
                        try:
                            gaze[int(x / 16)][int(y / 16)] += lens[i]
                        except:
                            pass
                    gaze = (gaze - np.min(gaze)) / (np.max(gaze) - np.min(gaze))
                    data[stim][participant] = np.array(gaze, dtype=np.float64)
        return data

    def load_stim(self, stim_folder):
        pics = {}
        for pic in self.human_coords:
            im_path = f'{stim_folder}/{pic[:-4]}_smap.png'
            if os.path.isfile(im_path):
                eml_net = imread(im_path)
                eml_net = eml_net[::16, ::16]
                eml_net = (eml_net - np.min(eml_net)) / (np.max(eml_net) - np.min(eml_net))
                pics[pic] = {'flat': eml_net.flatten(), 'map': eml_net, 'salient': len(eml_net[eml_net > 0.6])}
        return pics

    # def find_pareto_efficient(self, costs):
    #     best_inds = np.arange(costs.shape[0])

    #     for i, c in enumerate(costs):
    #       best_inds[i] = sum([list(p[:,j]).index(i) for j in len(costs[0])], [])

    #     return best_inds.argsort()
    best_inds=[]
    costs=[]
    def find_pareto_efficient(self, costs):
        p = costs.argsort(0)
        best_inds = np.ones(len(p))

            # Check if costs array is empty, return an empty array
        if len(costs) == 0:
          return np.array([])
        for i in range(len(p)):
            best_inds[i] = sum([list(p[:,j]).index(i) for j in range(len(costs[0]))])
        best_inds = best_inds.argsort()

        # # fill with other variants if there are <3 optimal
        # if len(best_inds) < self.n_desc():
        #     fits_sorted = np.array([ind for ind in np.mean(costs, axis=1).argsort() if ind not in best_inds])
        #     best_inds = np.r_[best_inds, fits_sorted[:self.n_desc - len(best_inds)]]
        return best_inds


    def calculate_fitness(self, parameters, stim):
        all_coords,ad_stat, all_rt, all_ks, all_aj, all_human_rt = [], [], [], [], [],[]

        input_seq = []

        for stim_key in list(self.stim.keys()):
            salient = self.stim[stim_key]['salient']

            T = 20
            input_seq = np.tile(self.stim[stim_key]['flat'], (T, 1))

            threshold = self.threshold + salient * parameters[7]
            print('new thresh ' + str(threshold))

            if self.lca_model == 'local':
                lca = SLCA_local(n_units=self.n_units, dt_t=self.dt_t, leak=parameters[0], competition=parameters[1],
                                self_excit=parameters[2], w_input=parameters[3], w_cross=parameters[4], offset=parameters[5],
                                noise_sd=parameters[6].astype(np.float64), threshold=threshold)
            elif self.lca_model == 'global':
                lca = SLCA_global(n_units=self.n_units, dt_t=self.dt_t, leak=parameters[0], competition=parameters[1],
                                  self_excit=parameters[2], w_input=parameters[3], w_cross=parameters[4], offset=parameters[5],
                                  noise_sd=parameters[6].astype(np.float64), threshold=threshold)

            print(mp.current_process().name + ' LCA started ' + stim_key + ' ' + ', '.join(
                [str(p) for p in parameters]) + ' ' + str(threshold))
            coords, rt = lca.run_2dim(stimuli=input_seq)
            import gc;
            gc.collect()
            print(mp.current_process().name + ' LCA finished ' + stim_key + ' ' + ', '.join([str(p) for p in rt]))

            trial_coord = np.array(coords, dtype=np.float64)
            trials_h = list(trial_coord // 120)
            trials_w = list(trial_coord % 120)

            lca_map = np.zeros(shape=(68, 120), dtype=np.float64)
            for i in range(len(rt)):
                lca_map[int(trials_h[i]), int(trials_w[i])] = rt[i]
            lca_smap = gaussian_filter(lca_map, sigma=1.5)
            lca_smap = (lca_smap - np.min(lca_smap)) / (np.max(lca_smap) - np.min(lca_smap))

            all_coords.append(coords)
            all_rt.extend(rt)

            participant = list(self.human_rt[stim_key].keys())[self.part]
            # for participant in self.human_rt[stim]:
            if 2 > 1:
                if len(self.human_rt[stim_key][participant]) == 0:
                    print(f'NO STIM {stim_key} for participant {participant}')
                else:
                    try:
                        ks = stats.ks_2samp(self.human_rt[stim_key][participant], rt)
                    except Exception as e:
                        ks = None
                        print(
                            f'ks stim {stim_key}, participant {participant}:: {str(e)}. RT {rt}, human RT {self.human_rt[stim_key][participant]}')

                    if ks and ks.statistic:
                        all_ks.append(ks.statistic)
                    else:
                        all_ks.append(0.99999)

                    all_human_rt.extend(self.human_rt[stim_key][participant])

                    if participant in self.human_coords[stim_key]:
                        try:
                            aj = auc_judd(lca_smap, self.human_coords[stim_key][participant])
                            print(
                                f'aj GOOD stim {stim_key}, participant {participant}:: SM max {np.max(lca_smap)}, min: {np.min(lca_smap)}, mean {np.mean(lca_smap)}')
                        except Exception as e:
                            aj = 0.00001
                            print(
                                f'aj stim {stim_key}, participant {participant}:: {str(e)}. SM max {np.max(lca_smap)}, min {np.min(lca_smap)}, mean {np.mean(lca_smap)}')

                    print(
                        f'stim {stim_key}, participant {participant}, params {parameters}:: ks {ks}, 1-aj {1 - aj}')
                    all_aj.append(1 - aj)

                    # Anderson-Darling test

                    if participant in self.human_coords[stim_key]:
                        try:
                            aj = auc_judd(lca_smap, self.human_coords[stim_key][participant])
                            print(
                                f'aj GOOD stim {stim_key}, participant {participant}:: SM max {np.max(lca_smap)}, min: {np.min(lca_smap)}, mean {np.mean(lca_smap)}')

                            # Anderson-Darling test
                            ad_stat, ad_critical_values, ad_significance_level = anderson(rt, dist='norm')
                            print(
                                f'Anderson-Darling stat: {ad_stat}, critical values: {ad_critical_values}, significance level: {ad_significance_level}')

                        except Exception as e:
                            aj = 0.00001
                            print(
                                f'aj stim {stim_key}, participant {participant}:: {str(e)}. SM max {np.max(lca_smap)}, min {np.min(lca_smap)}, mean {np.mean(lca_smap)}')

                    print(
                        f'stim {stim_key}, participant {participant}, params {parameters}:: ks {ks}, 1-aj {1 - aj}, AD stat {ad_stat}')

                    all_aj.append(1 - aj)

        all_human_rt = np.random.choice([el for el in all_human_rt if 50 <= el <= 750], 50)
        if len(all_rt) > 50:
            all_rt = np.random.choice(all_rt, 50)

        ks_fin = stats.ks_2samp(all_human_rt, all_rt).statistic
        res = {'params': parameters, 'fitness': (np.mean(all_ks), np.mean(all_aj))}
        return res


    def process_fit(self, param_set, output, idx):
        print(f'{mp.current_process().name} STARTED params_set {idx}')
        res = self.calculate_fitness(param_set,self.stim)
        print(f'{mp.current_process().name} FINISHED params_set {idx}')
        output.put(res)
        return res

    def select_best(self, sets):
        output = mp.Queue()

        processes = []

        for i in range(len(sets)):
            p = mp.Process(target=self.process_fit, args=(sets[i], output, i))
            processes.append(p)
            print(f'{p.name} started')
            p.start()
        for p in processes:
            p.join()

        results = []
        while not output.empty():
            o = output.get()
            results.append(o)

        fits = np.array([el['fitness'] for el in results])
        print(f'fits {len(fits)} {fits}')

        # find pareto optimal
        print('started computing pareto')
        best_inds = self.find_pareto_efficient(fits)
        print('best_pareto ' + ' '.join([str(i) for i in best_inds]))

        # fill with other variants if there are <3 optimal
        if len(best_inds) < 3:
            fits_sorted = np.array([ind for ind in np.mean(fits, axis=1).argsort() if ind not in best_inds])
            best_inds = np.r_[best_inds, fits_sorted[:3]]

        results = np.array(results)
        print(f'BEST RES: {results[best_inds]}')
        return np.array([res['params'] for res in results[best_inds]])

        #       # find best descendants
        # if self.n_desc == 1:
        #     best_inds = fits.argsort()[-self.n_desc:][::-1]
        # else:
        #     print('started computing pareto')
        #     best_inds = self.find_pareto_efficient(fits)[:self.n_desc]


    def random_gen(self, sets_num):
        gen = np.array([np.random.uniform(low=self.params_range[d][0], high=self.params_range[d][1], size=sets_num) for d in self.params_range]).T
        gen[:,list(self.fixed.keys())] = list(self.fixed.values())
        return gen
    def first_gen(self):
        return self.random_gen(self.gen_size)

    def mutate(self, set_params):
        param_num = np.random.choice(self.unfixed_inds)
        new_params = set_params.copy()
        import pdb; pdb.set_trace()
        new_params[param_num] *= np.random.choice([0.95, 1.05])
        if new_params[param_num] < self.params_diap[param_num][0]:
            new_params[param_num] = self.params_diap[param_num][0]
        elif new_params[param_num] > self.params_diap[param_num][1]:
            new_params[param_num] = self.params_diap[param_num][1]
        print("param_num:", param_num)
        print("length of self.params_diap:", len(self.params_diap))

        return new_params


    def crossover(self, parent0, parent1):
        cross_ind = np.random.choice(self.unfixed_inds)
        child = parent0.copy()
        child[cross_ind:] = parent1[cross_ind:]
        return child

    def next_gen(self, g, all_sets):
        print(f'GENERATION {g}')
        print('REMOVING DUPLICATES')
        unq, count = np.unique(all_sets, axis=0, return_counts=True)
        repeated_groups = unq[count > 1]
        for repeated_group in repeated_groups:
            repeated_idx = np.argwhere(np.all(all_sets == repeated_group, axis=1))
            repeated_idx = repeated_idx.ravel()[1:]
            all_sets[repeated_idx] = np.apply_along_axis(self.mutate, 1, all_sets[repeated_idx])

        print('SELECTION')
        gens_best = self.select_best(all_sets)
        print(f'gens_best shape, {gens_best.shape}')

        print('MUTATION')
        gens_mutated = np.apply_along_axis(self.mutate, 1, gens_best)
        print(f'mutated shape, {gens_mutated.shape}')

        print('CROSSOVER')
        parents0 = gens_best.copy()
        parents1_inds = [np.random.choice(np.delete(np.arange(len(gens_best)), [i]))
                         for i in range(len(gens_best))]
        parents1 = gens_best[parents1_inds]

        gens_crossover = np.array([self.crossover(parents0[i], parents1[i]) for i in range(len(parents0))])
        print(f'crossover shape, {gens_crossover.shape}')

        print('RANDOM')
        gens_random = self.random_gen(self.n_desc)
        print(f'random shape, {gens_random.shape}')

        gens = np.concatenate([gens_best, gens_mutated, gens_crossover, gens_random])
        print(f'all shape, {gens.shape}')
        return gens

    def initialize_lca_model(self, params, salient):
        # Implement your LCA model initialization here based on the given parameters
        # Return the initialized LCA model, salient value, and input sequence
        threshold = self.threshold + salient * params[7]

        if self.lca_model == 'local':
            lca = SLCA_local(n_units=self.n_units, dt_t=self.dt_t, leak=params[0], competition=params[1],
                             self_excit=params[2], w_input=params[3], w_cross=params[4], offset=params[5],
                             noise_sd=params[6].astype(np.float64), threshold=threshold)
        elif self.lca_model == 'global':
            lca = SLCA_global(n_units=self.n_units, dt_t=self.dt_t, leak=params[0], competition=params[1],
                              self_excit=params[2], w_input=params[3], w_cross=params[4], offset=params[5],
                              noise_sd=params[6].astype(np.float64), threshold=threshold)

        return lca, salient

    def compute_saliency_map(self, lca_model, salient,input_seq):
        # Implement your saliency map computation using the initialized LCA model
        # Return the computed saliency map
        coords, rt = lca_model.run_2dim(stimuli=self.input_seq)
        import gc
        gc.collect()

        trial_coord = np.array(coords, dtype=np.float32)
        trials_h = list(trial_coord // 120)
        trials_w = list(trial_coord % 120)

        lca_map = np.zeros(shape=(68, 120), dtype=np.float32)
        for i in range(len(rt)):
            lca_map[int(trials_h[i]), int(trials_w[i])] = rt[i]
        lca_smap = gaussian_filter(lca_map, sigma=1.5)
        lca_smap = (lca_smap - np.min(lca_smap)) / (np.max(lca_smap) - np.min(lca_smap))

        return lca_smap

    # Function to visualize the salience map
    def visualize_salience_map(self,saliency_map, output_path='salience_map.png'):
        # Visualize and save the salience map
        plt.imshow(saliency_map, cmap='viridis')
        plt.title('Saliency Map')
        plt.savefig(output_path)
        plt.show()

    # Function to visualize human gaze prediction
    def visualize_human_gaze(image_path, saliency_map, human_gaze_coordinates, output_path='human_gaze_prediction.png'):
        # Load the image
        image = imread(image_path)

        # Plot the image
        plt.imshow(image)

        # Plot the saliency map
        plt.imshow(saliency_map, cmap='viridis', alpha=0.5, origin='lower')

        # Plot human gaze coordinates
        plt.scatter(human_gaze_coordinates[:, 1], human_gaze_coordinates[:, 0], c='red', marker='x')

        # Set title and show the plot
        plt.title('Human Gaze Prediction')
        plt.savefig(output_path)
        plt.show()


# Train

In [14]:
import argparse
import logging
import matplotlib.pyplot as plt
from genetic import *
from slca import *
from utils import*

data_folder = '/content/drive/MyDrive/slca-main-pavan/data'
parameter_folder = '/content/drive/MyDrive/slca-main-pavan/config'
#data_folder = '/content/drive/MyDrive/Colab-Notebooks/slca_project/slca-main-pavan/data'
#parameter_folder = '/content/drive/MyDrive/Colab-Notebooks/slca_project/slca-main-pavan/config'

In [15]:
basic_loader = ParamsLoader(parameter_folder)
#parse the parameters of the genetic algorithm
gens_num, gen_size, model_type, participants, metrics, n_metrics, lca_model = GA_ParamsLoader(parameter_folder).load('ga_parameters')
    #parameters of simulation
trial_length, n_trials, desired_res = basic_loader.load('slca_parameters_sim').values()
data = DataLoader(data_folder, desired_res)
#parse the parameters range of the SLCA
params_range = SLCA_ParamsRangeLoader(parameter_folder).load('slca_parameters_range')

#parse the initial parameters of the SLCA
params_init = SLCA_ParamsInitLoader(parameter_folder).load('slca_parameters_init')

#parse fixed parameters of the SLCA
fixed_parameters = SLCA_ParamsFixedLoader(parameter_folder).load('slca_parameters_fixed')

In [17]:

def configure_logging():
    logging.basicConfig(filename='ga_log.txt', level=logging.INFO)

In [18]:
def visualize_human_gaze(image_path, saliency_map, human_gaze_coordinates, output_path='human_gaze_prediction.png'):
    # Load the image
    image = imread(image_path)

    # Plot the image
    plt.imshow(image)

    # Plot the saliency map
    plt.imshow(saliency_map, cmap='viridis', alpha=0.5, origin='lower')

    # Plot human gaze coordinates
    plt.scatter(human_gaze_coordinates[:, 1], human_gaze_coordinates[:, 0], c='red', marker='x')

    # Set title and show the plot
    plt.title('Human Gaze Prediction')
    plt.savefig(output_path)
    plt.show()

In [19]:
configure_logging()

# parser = argparse.ArgumentParser()
# parser.add_argument('--model', '-m', type=str, default='local', help='local or global model')
# parser.add_argument('--participant', '-p', type=int, default=0, help='participant number')
# parser.add_argument('--data', '-d', type=str, default='data', help='path to data folder')
# args = parser.parse_args()
model_type, participant, data_folder = "local",0 , "data"

n_units = 8160
time = 700
project_path = r"/content/drive/MyDrive/slca-main-pavan"
#project_path = r"/content/drive/MyDrive/Colab-Notebooks/slca_project/slca-main-pavan"
data_folder = f'{project_path}/data'
stim_path = f'{project_path}/data/smaps/eml-net1'
human_rt_path = f'{project_path}/data/all_rts.json'
human_coords_path = f'{project_path}/data/all_coords.json'
image_folder = f'{project_path}/data/smaps/eml-net1'
gens_num = 2
params = np.zeros(shape=(gens_num, 12, 8))

if model_type == 'local':
    params_first = np.array([[0.256, 0.5, 0.37, 0.64, 0.097, 0.31, 1.043, 0.465],
                              [0.256, 0.9, 0.25, 0.1, 0.097, 0.31, 3.0, 0.465],
                              [0.256, 0.8, 0.25, 0.1, 0.097, 0.31, 4.0, 0.2]])
    params_diap = ((0.1, 0.5), (0.0005, 1.0), (0.05, 0.5), (0.1, 0.9), (0.2, 1.0), (0.0, 10.0), (0.2, 5.0), (0.05, 1.5))
elif model_type == 'global':
    params_first = np.array([[0.4, 0.024, 0.4, 0.1, 1.0, 0.1, 1., 0.178],
                              [0.3, 0.003, 0.4, 0.2, 0.8, 0.1, 2., 0.5],
                              [0.25, 0.01, 0.4, 0.1, 0.9, 0.2, 3., 0.3]])
    params_diap = ((0.1, 0.5), (0.0005, 0.1), (0.05, 0.5), (0.1, 0.9), (0.2, 1.0), (0.0, 10.0), (0.2, 5.0), (0.05, 1.5))

ga = GA(lca_model=model_type,gen_size=gen_size, time=time, n_units=n_units,fixed=fixed_parameters,params_range=params_range, stim_path=stim_path, human_rt_path=human_rt_path, human_coords_path=human_coords_path, params_diap=params_diap, part=participant)
#initial parameter sets
params = np.zeros(shape=(gens_num, gen_size, len(basic_loader.all_params)))
#params = np.zeros((gens_num, gen_size, len(params_range)))
print('init_params', params.shape)
params[0] = ga.first_gen()

for i, param_set in enumerate(params_init):
    params[0, i, list(param_set.keys())] = list(param_set.values())

for g in range(1, gens_num):  # Start from 1 for next_gen
        logging.info(f'GENERATION {g}/{gens_num}')

        #params[g],gens_best = ga.next_gen(g, params[g-1])
        all_gens, gens_best = ga.next_gen(g, params[g - 1])
        params[g] = gens_best
        print(f'Shape of params[{g}]: {params[g].shape}')
        print(f'Shape of next_gen result: {all_gens.shape}')


        logging.info(f'GENERATION {g} RES {list(params[g])}')

init_params (2, 3, 10)
GENERATION 1
REMOVING DUPLICATES
SELECTION
Process-1 started
Process-2 started
Process-1 STARTED params_set 0Process-3 started

Process-2 STARTED params_set 1
new thresh 103.0
new thresh 56.107
Process-3 STARTED params_set 2
new thresh 201.0
Process-2 LCA started childroom01.jpg 0.01, 0.256, 0.9, 0.25, 0.097, 0.1, 0.31, 2.0, 24.258772873077632, 0.465 103.0


  0%|          | 0/20 [00:00<?, ?it/s]

Process-1 LCA started childroom01.jpg 0.01, 0.256, 0.5, 0.37, 0.097, 0.64, 0.31, 1.043, 17.311783848922545, 0.465 56.107


  0%|          | 0/20 [00:00<?, ?it/s]

Process-3 LCA started childroom01.jpg 0.01, 0.256, 0.8, 0.25, 0.097, 0.1, 0.31, 4.0, 14.045300964154606, 0.2 201.0
STARTED trial/Total: 0/20


...

STARTED trial/Total: 18/20


 95%|█████████▌| 19/20 [00:31<00:01,  1.78s/it]

STARTED trial/Total: 19/20


100%|██████████| 20/20 [00:32<00:00,  1.62s/it]


Process-3 LCA finished child2.jpg 19.0, 18.0, 18.0, 18.0, 18.0, 19.0, 18.0, 18.0, 18.0, 18.0, 17.0, 19.0, 18.0, 18.0, 18.0, 18.0, 17.0, 18.0, 19.0, 18.0
aj GOOD stim child2.jpg, participant LMe_2.asc:: SM max 1.0, min: 0.0, mean 0.02643478466789675
stim child2.jpg, participant LMe_2.asc, params [1.0000000e-02 2.5600000e-01 8.0000000e-01 2.5000000e-01 9.7000000e-02
 1.0000000e-01 3.1000000e-01 4.0000000e+00 1.4045301e+01 2.0000000e-01]:: ks KstestResult(statistic=1.0, pvalue=6.223019973618864e-15, statistic_location=19.0, statistic_sign=-1), 1-aj 0.455676065
aj GOOD stim child2.jpg, participant LMe_2.asc:: SM max 1.0, min: 0.0, mean 0.02643478466789675
Anderson-Darling stat: 2.889037063269811, critical values: [0.506 0.577 0.692 0.807 0.96 ], significance level: [15.  10.   5.   2.5  1. ]
stim child2.jpg, participant LMe_2.asc, params [1.0000000e-02 2.5600000e-01 8.0000000e-01 2.5000000e-01 9.7000000e-02
 1.0000000e-01 3.1000000e-01 4.0000000e+00 1.4045301e+01 2.0000000e-01]:: ks Kstest

In [20]:
run_command = f"measure.py"
%run {run_command}

In [21]:
import json
import math


def calculate_reaction_time(coordinates):
    reaction_times = []

    for key, points in coordinates.items():
        # Ensure that there are at least four points to calculate reaction time
        if len(points) > 4:
            print(f"Skipping {key} due to insufficient points.")
            continue

        for i in range(2, len(points), 2):
            delta_x = points[i] - points[i - 2]
            delta_y = points[i + 1] - points[i - 1]
            distance = math.sqrt(delta_x**2 + delta_y**2)
            reaction_time = 0.5 * distance
            reaction_times.append((key, reaction_time))

    return reaction_times

with open('/content/drive/MyDrive/slca-main-pavan/data/all_rts.json', 'r') as file:
     coordinates = json.load(file)

print(coordinates)
reaction_times = calculate_reaction_time(coordinates)

# Print or use reaction_times as needed
for key, reaction_time in reaction_times:
   print(f"Image: {key}, Reaction Time: {reaction_time:.2f} units")

{'kitchen265.jpg': {'LMe_1.asc': [127.0, 172.0, 230.0, 199.0, 349.0, 396.0, 144.0, 243.0, 252.0, 149.0, 324.0, 223.0, 465.0, 116.0, 247.0, 110.0, 163.0, 216.0, 272.0, 104.0, 249.0, 263.0, 244.0, 276.0, 185.0, 402.0, 188.0, 236.0, 332.0, 205.0, 313.0, 155.0, 428.0, 120.0, 252.0, 116.0], 'LMe_37.asc': [394.0, 279.0, 250.0, 217.0, 259.0, 179.0, 437.0, 337.0, 215.0, 141.0, 262.0, 135.0, 267.0, 444.0, 200.0, 302.0, 303.0, 249.0, 378.0, 279.0, 436.0, 315.0, 199.0, 176.0, 264.0, 153.0, 560.0, 441.0, 215.0, 120.0, 459.0], 'LMe_4.asc': [123.0, 159.0, 180.0, 174.0, 194.0, 198.0, 147.0, 243.0, 247.0, 222.0, 881.0, 180.0, 590.0, 264.0, 243.0, 360.0, 149.0, 156.0, 129.0, 285.0, 325.0, 372.0, 374.0, 272.0, 532.0, 205.0, 246.0, 148.0, 773.0, 188.0, 169.0], 'LMe_40.asc': [238.0, 187.0, 431.0, 217.0, 224.0, 223.0, 278.0, 241.0, 134.0, 305.0, 254.0, 322.0, 432.0, 341.0, 317.0, 297.0, 346.0, 288.0, 394.0, 329.0, 75.0, 230.0, 392.0, 254.0, 108.0, 248.0, 252.0, 231.0, 200.0, 134.0, 363.0, 225.0, 306.0, 69.

In [22]:
print(reaction_times)

[]


In [23]:
import json
import math

def calculate_stimulated_reaction_time(coordinates):
    reaction_times = []

    for key, points_list in coordinates.items():
        # Ensure that there are at least four points to calculate reaction time
        if len(points_list) < 8:
            print(f"Skipping {key} due to insufficient points.")
            continue

        # Calculate the reaction time for each pair of consecutive points
        for i in range(4, len(points_list), 4):
            delta_x = points_list[i] - points_list[i - 4]
            delta_y = points_list[i + 1] - points_list[i - 3]
            distance = math.sqrt(delta_x**2 + delta_y**2)
            reaction_time = 0.5 * distance
            reaction_times.append((key, reaction_time))

    return reaction_times

with open('/content/drive/MyDrive/slca-main-pavan/data/all_rts.json', 'r') as file:
    coordinates = json.load(file)

print(coordinates)
stimulated_reaction_times = calculate_stimulated_reaction_time(coordinates)

# Print or use stimulated_reaction_times as needed
for key, reaction_time in stimulated_reaction_times:
    print(f"Image: {key}, Stimulated Reaction Time: {reaction_time:.2f} units")


{'kitchen265.jpg': {'LMe_1.asc': [127.0, 172.0, 230.0, 199.0, 349.0, 396.0, 144.0, 243.0, 252.0, 149.0, 324.0, 223.0, 465.0, 116.0, 247.0, 110.0, 163.0, 216.0, 272.0, 104.0, 249.0, 263.0, 244.0, 276.0, 185.0, 402.0, 188.0, 236.0, 332.0, 205.0, 313.0, 155.0, 428.0, 120.0, 252.0, 116.0], 'LMe_37.asc': [394.0, 279.0, 250.0, 217.0, 259.0, 179.0, 437.0, 337.0, 215.0, 141.0, 262.0, 135.0, 267.0, 444.0, 200.0, 302.0, 303.0, 249.0, 378.0, 279.0, 436.0, 315.0, 199.0, 176.0, 264.0, 153.0, 560.0, 441.0, 215.0, 120.0, 459.0], 'LMe_4.asc': [123.0, 159.0, 180.0, 174.0, 194.0, 198.0, 147.0, 243.0, 247.0, 222.0, 881.0, 180.0, 590.0, 264.0, 243.0, 360.0, 149.0, 156.0, 129.0, 285.0, 325.0, 372.0, 374.0, 272.0, 532.0, 205.0, 246.0, 148.0, 773.0, 188.0, 169.0], 'LMe_40.asc': [238.0, 187.0, 431.0, 217.0, 224.0, 223.0, 278.0, 241.0, 134.0, 305.0, 254.0, 322.0, 432.0, 341.0, 317.0, 297.0, 346.0, 288.0, 394.0, 329.0, 75.0, 230.0, 392.0, 254.0, 108.0, 248.0, 252.0, 231.0, 200.0, 134.0, 363.0, 225.0, 306.0, 69.

In [25]:
import numpy as np
from measure import Measure, KolmogorovSmirnov, AucJudd


simulated_reaction_times = human_rt_path
human_reaction_times = coordinates

simulated_saliency_map = np.array( [[910.9, 538.8], [512.2, 659.8], [499.8, 926.4], [546.3, 794.0], [542.3, 938.6], [527.7, 838.1], [548.4, 773.0], [580.4, 687.4],
                                    [514.3, 629.0], [490.1, 579.6], [245.1, 458.8], [182.4, 452.9], [467.0, 427.2], [752.6, 440.7], [735.6, 463.3], [841.6, 456.3],
                                    [705.1, 438.8], [756.3, 525.5], [899.5, 556.0], [1165.3, 558.1], [869.0, 462.8], [910.7, 432.8], [1247.5, 464.0],
                                    [1352.9, 460.5], [1545.8, 438.7], [1575.5, 427.7], [1745.7, 479.3], [1715.2, 473.2], [1603.6, 967.7], [1558.1, 954.1], [1177.7, 579.2]])
ground_truth_coords = np.array([[890.3, 525.6], [478.2, 430.6], [181.4, 451.0], [264.4, 467.8], [180.8, 440.3], [1002.6, 472.2], [853.1, 572.1], [519.0, 838.9], [451.1, 916.2], [998.7, 539.7],
                                [1330.7, 471.9], [1593.2, 438.9], [1713.1, 474.8], [1701.0, 487.2], [886.1, 477.9], [767.8, 483.7], [932.1, 493.9], [914.2, 571.9], [1162.0, 581.0], [1193.9, 589.1],
                                [1008.1, 475.8], [842.7, 472.7], [873.1, 476.0], [1069.8, 474.3], [1235.9, 478.9], [1335.9, 465.7], [848.3, 185.7], [428.4, 491.9],
                                [513.1, 575.0], [565.2, 640.9], [509.0, 763.2], [580.8, 662.3], [555.5, 745.8], [1493.4, 603.0], [1553.0, 830.4], [1317.3, 901.2]],)

# Instantiate measurement classes with your data
ks_measure = KolmogorovSmirnov(sim_rt=simulated_reaction_times, human_rt=human_reaction_times)
auc_judd_measure = AucJudd(sim_smap=simulated_saliency_map, human_coords=ground_truth_coords)

# Run the measurements
ks_result = ks_measure.run()
auc_judd_result = auc_judd_measure.run()

# Print or use the results as needed
print("Kolmogorov-Smirnov Result:", ks_result)
print("AUC Judd Result:", auc_judd_result)


Kolmogorov-Smirnov Result: None
AUC Judd Result: None


In [46]:
run_command = f"genetic.py"
%run {run_command}

In [ ]:
import numpy as np

fits = np.array([[1.         ,0.83332667],
 [1.      ,0.99999   ],
 [1.      ,0.83332667],
 [1.      ,0.6596603 ],
 [1.      ,0.66310778],
 [1.      ,0.49275853],
 [1.      ,0.49063837],
 [1.      ,0.49086869],
 [1.      ,0.50069545]])



def find_pareto_efficient(costs):
   p = costs[:].argsort(0)
   best_inds = np.ones(len(p), dtype=np.int64)
   for i in range(len(p)):
       best_inds[i] = sum([list(p[:, j]).index(i) for j in range(len(costs[0]))])
   best_inds = best_inds.argsort()
   return best_inds

find_pareto_efficient(fits)

array([0, 6, 3, 5, 7, 1, 2, 4, 8])

In [57]:
import numpy as np

fits = np.array([[1.        , 0.83332667],
 [1.        , 0.83332667],
 [1.        , 0.83332667],
 [1.        , 0.66363707],
 [1.        , 0.49086869],
 [1.        , 0.50352471],
 [1.        , 0.49292828],
 [1.        , 0.50780851],
 [1.        , 0.48974423],
 [1.        , 0.43692392],
 [0.99074074, 0.42283899],
 [0.96296296, 0.46105233]])

def find_pareto_efficient(costs):
  p = costs[:].argsort(0)
  best_inds = np.ones(len(p), dtype=np.int64)
  for i in range(len(p)):
      best_inds[i] = sum([list(p[:, j]).index(i) for j in range(len(costs[0]))])
  best_inds = best_inds.argsort()
  return best_inds

find_pareto_efficient(fits)

array([10, 11,  4,  0,  9,  1,  3,  5,  6,  8,  2,  7])

In [ ]:
gens_best

array([[0.83332667, 0.83332667, 0.83332667, 0.66363707, 0.49086869,
        0.50352471, 0.49292828, 0.50780851, 0.48974423, 0.43692392,
        0.42283899, 0.46105233]])

In [ ]:
run_command = f"{project_path}/src/train.py"
%run {run_command}

GENERATION 0/2
GENERATION 0
REMOVING DUPLICATES
SELECTION
Process-13 started
Process-14 started
Process-13 STARTED params_set 0Process-15 started

Process-14 STARTED params_set 1new thresh 5.0Process-16 started
Process-17 started

Process-15 STARTED params_set 2
new thresh 7.45
Process-18 started

new thresh 7.45Process-16 STARTED params_set 3Process-19 started


new thresh 7.45Process-17 STARTED params_set 4
Process-20 started

new thresh 5.0
Process-18 STARTED params_set 5
new thresh 7.45Process-21 started
Process-19 STARTED params_set 6

new thresh 5.0Process-22 started

Process-20 STARTED params_set 7
new thresh 7.45
Process-21 STARTED params_set 8
new thresh 5.0Process-23 started

Process-22 STARTED params_set 9
Process-24 started
new thresh 7.45
Process-23 STARTED params_set 10
new thresh 5.0
Process-24 STARTED params_set 11
new thresh 7.45
Process-13 LCA started childroom01.jpg 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0 5.0


  0%|          | 0/40 [00:00<?, ?it/s]

Process-14 LCA started childroom01.jpg 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05 7.45
Process-15 LCA started childroom01.jpg 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05 7.45


  0%|          | 0/40 [00:00<?, ?it/s]

Process-20 LCA started childroom01.jpg 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05 7.45
Process-18 LCA started childroom01.jpg 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.05 7.45


...

STARTED trial/Total: 1/40STARTED trial/Total: 1/40

STARTED trial/Total: 1/40
STARTED trial/Total: 1/40
STARTED trial/Total: 1/40
STARTED trial/Total: 1/40
STARTED trial/Total: 1/40
STARTED trial/Total: 1/40
STARTED trial/Total: 1/40
STARTED trial/Total: 1/40


  2%|▎         | 1/40 [05:10<3:21:46, 310.42s/it]


  2%|▎         | 1/40 [05:08<3:20:44, 308.84s/it]
Process Process-24:
Process Process-23:
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

  2%|▎         | 1/40 [05:08<3:20:30, 308.48s/it]

Process Process-13:
Process Process-16:
Process Process-18:
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

  2%|▎         | 1/40 [05:08<3:20:14, 308.05s/it]Process Process-17:

  2%|▎         | 1/40 [05:09<3:21:00, 309.25s/it]<generator object tqdm.__iter__ at 0x7bd421768040>Exception ignored in: 
<generator object tqdm.__iter__ at 0x7bd421633e60>
<generator object tqdm.__iter__ at 0x7bd421633e60>Process Process-20:

Traceback (most recent call last):
Traceback (most recent call last):

  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1197, in __iter__
Traceback (most recent call last):
  File "/usr/local/lib/python3

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2831, in safe_execfile
    py3compat.execfile(
  File "/usr/local/lib/python3.10/dist-packages/IPython/utils/py3compat.py", line 168, in execfile
    exec(compiler(f.read(), fname, 'exec'), glob, loc)
  File "/content/drive/MyDrive/slca-main-pavan/src/train.py", line 45, in <module>
    params[g] = ga.next_gen(g, params[g - 1])
  File "/content/drive/MyDrive/slca-main-pavan/src/genetic.py", line 229, in next_gen
    return new_params
  File "/content/drive/MyDrive/slca-main-pavan/src/genetic.py", line 172, in select_best
    return res
  File "/usr/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 43, in wait
    return self.poll(os.WNOHANG if timeout == 0.0 else 0)
  File "/usr/lib/python3.10/multiprocessing/popen_fork.py", line 27, in poll
    pid, sts

  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 462, in print_status
  2%|▎         | 1/40 [05:10<3:21:40, 310.26s/it]        Exception ignored in: : 
<generator object tqdm.__iter__ at 0x7bd421768040>fp_write('\n')fp_write('\r' + s + (' ' * max(last_len[0] - len_s, 0)))
Process Process-14:



Process Process-19:
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1288, in fp_write
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 455, in fp_write
      File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1197, in __iter__
        fp.write(str(s))self.fp.write(str(s))
self.close()
  File "/usr/local/lib/python3.10/dist-packages/tqdm/utils.py", line 195, in inner
  File "/usr/local/lib/python3.10/dist-packages/tqdm/utils.py", line 195, in inner

  File "/usr/local/lib/python3.10/dist-packages/tqdm/std.py", line 1303, in close
            return func(*args, **kwargs)self.display(pos=0)
  Fil

TypeError: ignored

In [ ]:
!cat /content/drive/MyDrive/slca-main/src/train.py